# Scrape LRcast webpages for episode info

In [36]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

NB: First episode currently at pg 51
Episode where youtube starts: 231, JOU (un)commons review

In [37]:
def getlrcontent(mysoup):
    lrcontent=[] # blank variable to store all entries
    for dum in mysoup.find_all('div', class_="entry-content"):
        lrcontent.append(dum.text)
    dum=lrcontent.find('\n\n')
    dum=lrcontent[dum:]
    x=dum.find('\xa0')
    dum=dum[:x]
    dum=dum.replace('\n','')
    return dum

def cleanlrcontent(currlrcontent):
    # clean up each row
    alldum=[]
    for n in np.arange(0,len(currlrcontent)):
        currcontent=currlrcontent[n]
        dum=currcontent.find('\n\n')
        dum=currcontent[dum:]
        x=dum.find('\xa0')
        dum=dum[:x]
        dum=dum.replace('\n','')
        alldum.append(dum)
    return alldum

In [38]:
# 'http://lrcast.com/episodes/page/51/' is the first page
lrpageurl='http://lrcast.com/episodes/page/51/'
print('First episode at {}'.format(lrpageurl))
numlrpages = 51

First episode at http://lrcast.com/episodes/page/51/


In [107]:
lrepisodes=pd.DataFrame([])
for n in np.arange(1,numlrpages+1):
    lrpageurl=('http://lrcast.com/episodes/page/{}/'.format(str(n)))
    print(lrpageurl)
    time.sleep(0.1)
    lrpage = requests.get(lrpageurl)
    lrpagetext = lrpage.text
    soup = BeautifulSoup(lrpagetext, 'lxml')
    
    # Get all titles for this page
    lrtitles=[] # blank variable to store all entries
    for dum in soup.find_all('h1', class_="entry-title"):
        dumtitle=dum.text
        dum=dumtitle[dumtitle.find('–')+2:]
        lrtitles.append(dum)
    
    # Get curr LR episode numbers
    lrnums=[]
    for dum in soup.find_all('h1', class_="entry-title"):
        dum = dum.text
        currepnum=(dum[dum.find('Resources ')+len('Resources '):dum.find('–')]).strip()
        if currepnum.isdigit():
            lrnums.append(int(currepnum))
        else:
            lrnums.append(9999)
            
    # Get all post dates for this page
    lrpostdates=[] # blank variable to store all entries
    for dum in soup.find_all('time', class_="entry-date"):
        lrpostdates.append(dum.text)
        
    # Get all content
    lrallcontent=[] # blank variable to store all entries
    for dum in soup.find_all('div', class_="entry-content"):
        lrallcontent.append(dum.text)

    # Get Episode URLs        
    lrurls=[] # blank variable to store all entries
    for dum in soup.find_all('h1', class_="entry-title"):
        for dum2 in dum.find_all('a', href=True):
            lrurls.append(dum2['href'])
    
    #lrcontent=cleanlrcontent(lrallcontent) # previously used this, but mght not be needed
    lrcontent=[] # minimal content variable to store all entries
    for dum in soup.find_all('div', class_="entry-content"):
        childdum = dum.find_all('p')
        dumm=childdum[0].text
        if len(dumm)<30:
            dumm=childdum[1].text
        lrcontent.append(dumm)
    
    currlrepisodes = pd.DataFrame(list(zip(lrnums, lrpostdates, lrtitles,lrurls,lrcontent,lrallcontent),),columns=['episodenum','postdate','title','URL','content','allcontent'],index=lrnums)
    lrframes = [lrepisodes,currlrepisodes]
    
    lrepisodes = pd.concat(lrframes)
lrepisodes.sort_index(inplace=True)
lrepisodes

http://lrcast.com/episodes/page/1/
http://lrcast.com/episodes/page/2/
http://lrcast.com/episodes/page/3/
http://lrcast.com/episodes/page/4/
http://lrcast.com/episodes/page/5/
http://lrcast.com/episodes/page/6/
http://lrcast.com/episodes/page/7/
http://lrcast.com/episodes/page/8/
http://lrcast.com/episodes/page/9/
http://lrcast.com/episodes/page/10/
http://lrcast.com/episodes/page/11/
http://lrcast.com/episodes/page/12/
http://lrcast.com/episodes/page/13/
http://lrcast.com/episodes/page/14/
http://lrcast.com/episodes/page/15/
http://lrcast.com/episodes/page/16/
http://lrcast.com/episodes/page/17/
http://lrcast.com/episodes/page/18/
http://lrcast.com/episodes/page/19/
http://lrcast.com/episodes/page/20/
http://lrcast.com/episodes/page/21/
http://lrcast.com/episodes/page/22/
http://lrcast.com/episodes/page/23/
http://lrcast.com/episodes/page/24/
http://lrcast.com/episodes/page/25/
http://lrcast.com/episodes/page/26/
http://lrcast.com/episodes/page/27/
http://lrcast.com/episodes/page/28/
h

,episodenum,postdate,title,URL,content,allcontent
1,1,"October 2, 2009",Introductions and Zendikar Draft Preview,http://lrcast.com/limited-resources-1-introduc...,"Your Limited Resources hosts, Marshall and Rya...","\n\n\nYour Limited Resources hosts, Marshall a..."
2,2,"October 9, 2009","Allies, Vampires, and Other Zendikar Annoyances",http://lrcast.com/limited-resources-2-allies-v...,More Zendikar! This week Marshall and Ryan do ...,"\n<!--\r\ngoogle_ad_client = ""ca-pub-662782498..."
3,3,"October 15, 2009",Bringing Home the Loot,http://lrcast.com/limited-resources-bringing-h...,More Zendikar! This week Marshall and Ryan do ...,\n\n\nMore Zendikar! This week Marshall and Ry...
4,4,"October 25, 2009",Breaking Bread,http://lrcast.com/limited-resources-4-breaking...,This week Marshall and Ryan talk about basic d...,\n\n\nThis week Marshall and Ryan talk about b...
5,5,"October 29, 2009",The Best-Case Scenario,http://lrcast.com/limited-resources-5-the-best...,This week Marshall and Ryan delve deep into a ...,\n\n\nThis week Marshall and Ryan delve deep i...
6,6,"January 5, 2009",Base Configuration,http://lrcast.com/limited-resources-6-base-con...,This week Ryan and Marshall talk about buildin...,\n\n\nThis week Ryan and Marshall talk about b...
6,6,"November 5, 2009",Base Configuration,http://lrcast.com/limited-resources-6-base-con...,This week Ryan and Marshall talk about buildin...,\n\n\nThis week Ryan and Marshall talk about b...
7,7,"November 12, 2009",Raiders of the Lost Archeytypes,http://lrcast.com/limited-resources-7-raiders-...,This week Ryan and Marshall finish up Quick Hi...,\n\n\nThis week Ryan and Marshall finish up Qu...
8,8,"November 19, 2009",Tightening Your Grasp,http://lrcast.com/limited-resources-8-tighteni...,"This week on episode 8 of Limited Resources, R...",\n\n\nThis week on episode 8 of Limited Resour...
9,9,"November 26, 2009",Clear Signals,http://lrcast.com/limited-resources-9-clear-si...,This week on Episode #9 of Limited Resources M...,\n\n\nThis week on Episode #9 of Limited Resou...


In [108]:
lrepisodes.head()

,episodenum,postdate,title,URL,content,allcontent
1,1,"October 2, 2009",Introductions and Zendikar Draft Preview,http://lrcast.com/limited-resources-1-introduc...,"Your Limited Resources hosts, Marshall and Rya...","\n\n\nYour Limited Resources hosts, Marshall a..."
2,2,"October 9, 2009","Allies, Vampires, and Other Zendikar Annoyances",http://lrcast.com/limited-resources-2-allies-v...,More Zendikar! This week Marshall and Ryan do ...,"\n<!--\r\ngoogle_ad_client = ""ca-pub-662782498..."
3,3,"October 15, 2009",Bringing Home the Loot,http://lrcast.com/limited-resources-bringing-h...,More Zendikar! This week Marshall and Ryan do ...,\n\n\nMore Zendikar! This week Marshall and Ry...
4,4,"October 25, 2009",Breaking Bread,http://lrcast.com/limited-resources-4-breaking...,This week Marshall and Ryan talk about basic d...,\n\n\nThis week Marshall and Ryan talk about b...
5,5,"October 29, 2009",The Best-Case Scenario,http://lrcast.com/limited-resources-5-the-best...,This week Marshall and Ryan delve deep into a ...,\n\n\nThis week Marshall and Ryan delve deep i...


In [ ]:
lrepisodes.to_csv('lr_eposides_master.csv')
lrepisodes.to_pickle('lr_eposides_master.pkl')